In [1]:
import mdptoolbox as mdpt, numpy as np
import mdptoolbox.example
import MDP

Suppose, given a transition function and discount rate, we generate a random reward function over all transitions. We then sparsify the reward function by setting some proportion (e.g. 10%) of the transition values to 0. We then generate the optimal policy for said reward function (using, for instance, policy iteration). We now attempt to build a model that can predict the sparsity used to generate the optimal policy given the transition function, discount rate, and policy itself, but *not* the reward function, as otherwise the problem would be trivial.

In [2]:
### Generate a bunch of MDPs with different parameters, sparsity

NUM_MDPs = 100
NUM_STATES = 100
NUM_ACTIONS = 4

def get_transition_matrix(num_states, num_actions, generator = np.random.dirichlet):
    """
    Returns a transition matrix for a given number of states and actions
    
    Returns:
        P: (num_actions, num_states, num_states) array, where P[a, s, s'] is the probability of 
        transitioning from state s to state s' given action a
    """
    P = np.zeros((num_actions, num_states, num_states)) # (A, S, S) shape
    for a in range(num_actions):
        for s in range(num_states):
            P[a, s, :] = generator(np.ones(num_states))
    return P

def get_reward_matrix(num_states, num_actions, sparsity = 0.0, generator = np.random.normal):
    """
    Returns a reward matrix for a given number of states and actions
    """
    R = np.zeros((num_states, num_actions))
    for a in range(num_actions):
        for s in range(num_states):
            if np.random.rand() < sparsity:
                R[s, a] = 0
            else:
                R[s, a] = generator()
    return R

DISCOUNT = 0.9
EPSILON = 0.01
MAX_ITER = 1000

The sparsity levels generated by generate_tests are divided using arange from 0 to 1 and then scrambled randomly, meaning that in effect each sparsity level in the training and test sets is sampled uniformly from [0, 1].

In [3]:
def generate_tests(num_mdps = NUM_MDPs, sparsity_levels = None, mdp_generator = mdpt.mdp.PolicyIteration):
    """
    Generate a bunch of MDPs with different sparsity levels, and return the sparsity levels and the MDPs

    Args:
        sparsity_levels: a list of sparsity levels to generate MDPs with
    Returns:
        sparsity_levels: the sparsity levels used to generate the MDPs, in the same order as the MDPs
        MDPS: an array of MDPs
    """
    sparsity_levels = sparsity_levels if sparsity_levels is not None else np.arange(num_mdps) / num_mdps
    sparsity_copy = sparsity_levels.copy() # defensive copy
    np.random.shuffle(sparsity_copy)
    MDPS = np.array([mdp_generator(
        get_transition_matrix(NUM_STATES, NUM_ACTIONS), 
        get_reward_matrix(NUM_STATES, NUM_ACTIONS, sparsity_copy[i]), 
        DISCOUNT, max_iter = MAX_ITER) 
        for i in range(num_mdps)
    ])
    return sparsity_copy, MDPS

sparsity_levels, MDPS = generate_tests()
for mdp in MDPS:
    mdp.run()
    # print(mdp.policy) # debug
# print(MDPS[0].policy) # debug

In [4]:
### Build a classifier to predict sparsity level from a policy
### Idea 1: hack-y heuristics

def heuristic_classifier(MDP, policy):
    """
    A heuristic classifier that predicts the sparsity level of an MDP's reward function given its 
    optimal policy
    1. 
    """
    # TODO: implement this


In [9]:
### Idea 2: neural network
# Thanks again ChatGPT for outlining the code structure

sparsity, MDPs = generate_tests(1000)
# print(np.array(MDPs[0].P).shape)
training_data = [(np.array(mdp.P), mdp.discount, mdp.policy, sparsity[i]) for i, mdp in enumerate(MDPs)]

from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

# Step 1: Feature extraction function
def extract_features(transition_function, discount_rate, optimal_policy):
    """
    Extract features from the MDP's transition function, discount rate, and optimal policy
    """
    # opt_policy = optimal_policy.reshape(-1, 1)  # Reshape for sklearn which expects 2D input

    # # Initialize the OneHotEncoder
    # encoder = OneHotEncoder(sparse=False)  # Use sparse=False to get a dense array

    # # Fit and transform
    # opt_policy_one_hot = encoder.fit_transform(opt_policy)
    features = np.concatenate((transition_function.flatten(), [discount_rate], optimal_policy.flatten()))
    # print(features.shape)
    # length 10*10*4 + 1 + 10 = 411

    # Placeholder features
    # features = np.random.rand(411)
    return features

# Step 2: Data preparation (assuming you have your data in an appropriate format)
# This is a placeholder function - you would replace it with actual data loading and processing
def prepare_data(training_data):
    features = []
    labels = []
    for transition_function, discount_rate, optimal_policy, sparsity_level in training_data:
        features.append(extract_features(transition_function, discount_rate, optimal_policy))
        labels.append(sparsity_level)
    return np.array(features), np.array(labels)

# Step 3: Model selection

def build_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='linear')  # Linear activation for regression output
    ])

    # Num parameters: 4011*64 + 64 + 64*64 + 64 + 64*64 + 64 + 64*1 + 1 = 265,153
    # Num data points: 10000
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error',  # Suitable for regression
                  metrics=['mae'])  # Mean Absolute Error as an additional metric
    # ``loss" refers to training data, ``val_loss" refers to validation data
    return model

features, labels = prepare_data(training_data)
# Example: features shape is (num_samples, num_features), adjust 'input_dim' accordingly
input_dim = features.shape[1]  # Assuming 'features' is already defined and preprocessed

model = build_model(input_dim)

# Training the model
model.fit(features, labels, epochs=100, validation_split=0.2, verbose = 1, 
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

# Don't forget to preprocess your new data before making predictions
# predicted_sparsity = model.predict(new_features)

# Step 4: Training the model (placeholder for training data)
# training_data = load_your_data_somehow()
# features, labels = prepare_data(training_data)
# model.fit(features, labels)

# Step 5: Prediction function
def predict_sparsity(transition_function, discount_rate, optimal_policy):
    features = extract_features(transition_function, discount_rate, optimal_policy).reshape(1, -1)
    predicted_sparsity = model(features) # more efficient than .predict() for single samples
    return predicted_sparsity

# Testing model
test_sparsity, test_MDPs = generate_tests()
test_data = [(np.array(mdp.P), mdp.discount, mdp.policy) for mdp in (test_MDPs)]
NUM_TESTS = 1000
mse = np.zeros(min(NUM_TESTS, len(test_data)))

for i in range(min(NUM_TESTS, len(test_data))):
    transition_function, discount_rate, optimal_policy = test_data[i]
    prediction = predict_sparsity(transition_function, discount_rate, optimal_policy)[0][0]
    mse[i] = (prediction - test_sparsity[i])**2
    print(f"Predicted sparsity level for MDP {i}: {prediction}, actual sparsity level: {test_sparsity[i]}, Squared error: {mse[i]}")

print(f"Mean squared error: {np.mean(mse)}")
print("Expected squared error: when x, y ~ U[0, 1], E[(x-y)^2] = 1/12 = 0.0833...")

Epoch 1/100
25/25 [==============================] - 3s 47ms/step - loss: 0.2056 - mae: 0.3619 - val_loss: 0.0569 - val_mae: 0.2046
Epoch 2/100
25/25 [==============================] - 1s 44ms/step - loss: 0.0539 - mae: 0.1812 - val_loss: 0.0118 - val_mae: 0.0895
Epoch 3/100
25/25 [==============================] - 2s 75ms/step - loss: 0.0376 - mae: 0.1530 - val_loss: 0.0189 - val_mae: 0.1217
Epoch 4/100
25/25 [==============================] - 2s 77ms/step - loss: 0.0285 - mae: 0.1315 - val_loss: 0.0273 - val_mae: 0.1485
Epoch 5/100
25/25 [==============================] - 2s 63ms/step - loss: 0.0249 - mae: 0.1252 - val_loss: 0.0180 - val_mae: 0.1167
Predicted sparsity level for MDP 0: 0.26282399892807007, actual sparsity level: 0.09, Squared error: 0.02986813336610794
Predicted sparsity level for MDP 1: 0.30783629417419434, actual sparsity level: 0.4, Squared error: 0.008494149893522263
Predicted sparsity level for MDP 2: 0.2889615595340729, actual sparsity level: 0.04, Squared error

With ten actions:
- As a control, when the input layer (with same dimension as transition_function + discount rate + optimal policy) is randomized, MSE = ~0.115
- I should also note that I'm choosing hyperparameters here in a rather unprincipled way by guess-timating their effects on the model
- The loss seems to settle around 0.033 after ~20% into each epoch when given 10^5 training points 

With 100 actions:
- Model does slightly better (now ~0.028); maybe patterns in MDP/sparsity become more apparent with more states?
- In terms of computation, generating the MDPs takes a lot longer than training the model

In [10]:
### Idea 3: Multiple linear regression 

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

sparsity, MDPs = generate_tests(1000)
# print(np.array(MDPs[0].P).shape)
training_data = [(np.array(mdp.P), mdp.discount, mdp.policy, sparsity[i]) for i, mdp in enumerate(MDPs)]
print(sparsity)
features, labels = prepare_data(training_data)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create a model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean squared error: {mse}")
print("Expected squared error: when x, y ~ U[0, 1], E[(x-y)^2] = 1/12 = 0.0833...")
print(f"Mean absolute error: {mae}")


[0.103 0.369 0.111 0.719 0.327 0.034 0.525 0.414 0.193 0.026 0.998 0.905
 0.294 0.841 0.114 0.519 0.944 0.005 0.014 0.697 0.118 0.123 0.594 0.415
 0.829 0.624 0.792 0.777 0.32  0.676 0.106 0.421 0.893 0.456 0.602 0.776
 0.312 0.495 0.405 0.176 0.5   0.05  0.137 0.686 0.044 0.206 0.825 0.87
 0.44  0.437 0.64  0.241 0.404 0.145 0.151 0.217 0.806 0.128 0.945 0.949
 0.451 0.692 0.989 0.936 0.231 0.423 0.319 0.51  0.856 0.372 0.723 0.371
 0.117 0.435 0.886 0.373 0.887 0.827 0.313 0.671 0.75  0.452 0.937 0.504
 0.446 0.916 0.465 0.959 0.168 0.309 0.375 0.66  0.115 0.007 0.629 0.076
 0.121 0.74  0.59  0.025 0.253 0.668 0.974 0.416 0.18  0.926 0.917 0.374
 0.902 0.731 0.342 0.049 0.275 0.971 0.367 0.636 0.052 0.984 0.718 0.346
 0.506 0.251 0.635 0.45  0.673 0.918 0.209 0.615 0.337 0.234 0.964 0.127
 0.659 0.287 0.746 0.13  0.767 0.996 0.029 0.544 0.302 0.757 0.281 0.116
 0.71  0.868 0.646 0.889 0.575 0.857 0.675 0.448 0.34  0.666 0.872 0.037
 0.993 0.227 0.616 0.381 0.388 0.427 0.264 0.412 0.2

- Interesting! When I increased the number of states from 10 to 100 and *decreased* the number of training data points from 10^5 to 10^4, test loss *decreased* from ~0.033 to ~0.014 and stayed that way with training data = 10^3